In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
####한국어 형태소 분석기####
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능
okt = Okt()

df_18=pd.read_csv('../원본데이터/news18.csv',encoding='cp949')
df_19=pd.read_csv('../원본데이터/news19.csv',encoding='cp949')
df_20=pd.read_csv('../원본데이터/news20.csv',encoding='cp949')
df_21=pd.read_csv('../원본데이터/news21.csv',encoding='cp949')

df_all=pd.concat([df_18,df_19,df_20,df_21])

df_all

df_all['time']=pd.to_datetime(df_all['time'])
df_all['datetime']=df_all['time'].dt.strftime('%Y-%m-%d')

df_all['datetime']=pd.to_datetime(df_all['datetime'])

df_new=df_all

df_new.reset_index(inplace=True)

df_new=df_new.groupby(['datetime'], as_index=False).agg(lambda x: ' '.join(set(x)))

df_new.head()

df_new

df_new.tail(1000)

week = [g for n, g in df_new.set_index('datetime').groupby(pd.Grouper(freq='W'))]

week[0]

C:\Users\jaemin\AppData\Local\Temp\ipykernel_2180\2809216454.py:23: FutureWarning: ['index', 'time'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_new=df_new.groupby(['datetime'], as_index=False).agg(lambda x: ' '.join(set(x)))


,theme,title,article
datetime,,,
2018-01-04,오늘의 뉴스,한돈 약세 상반기 내내 지속(1/4) 17년에도 한돈 5000원 못 이뤄(1/4) ...,"\t\t\t\t\t작년 돼지 출하 사상최고 기록\r\n1670만두, 전년비 1% 늘..."


### txt로 저장

In [3]:
error_index=[]
stem_list=[]

for index, text in enumerate(week):
    try:
        stem_list.append('{}번쨰 기사입니다.'.format(index))
        stem_list.append(text['article'][0])
    except:
        stem_list.append('NULL')
        error_index.append(index)
    with open('./news_stem/stem_list{}.txt'.format(index),'w',encoding='UTF-8') as f:
        f.write('\n'.join(map(str, stem_list)))

In [4]:
# for i in range(208):
#     f = open('./news_stem/stem_list{}.txt'.format(i), 'r',encoding='UTF-8')

#     text = f.read()
#     text_2 = text.replace("\n", "")
#     text_3 = text_2.replace("\t", "")
#     text_4 = text_3.replace("________________", "0")
#     text_5=text_4.replace('     ',"0")
#     content = text_5
#     print(content)
#     with open('./news_stem/stem_list2_{}.txt'.format(i),'w',encoding='UTF-8') as f:
#         f.write(content)
    
#     print("\n")

In [10]:
import json
import sys
import requests

file = open('./positive_negative_text_result2.txt', 'w', encoding='utf8') #결과파일
file2=open('./result.txt', 'w', encoding='utf8') #결과파일
n = 1 #첫번째
m = 800  #두번째
for i in range(1, 1000) : #41번
    client_id = "4q7skv8ste"  #클라이언트 키
    client_secret = "wmqQzlBqk2InZAdvDsHOHDZNIVO9PVBysFVxy8m4" #클라이언트 시크릿 키

    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }
#     for j in range(len(week)):
#         print("{}번째 기사입니다.".format(j)+"\n"+"\n"+"\n")
    f = open('./news_stem/stem_list207.txt', 'r',encoding='UTF-8')

    text = f.read()
    text_2 = text.replace("\n", "")
    text_3 = text_2.replace("\t", "")
    text_4 = text_3.replace("________________", "0")
    content = text_4[n:m] #800단어씩
    print(content)
    print("\n")

    data = {
      "content": content
    }
    print(json.dumps(data, indent=4, sort_keys=True))
    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code
    if(rescode == 200):
        print (response.text)
    else:
        print("Error : " + response.text)

    f.close()    

    text = response.json()

    raw = text['sentences']

    file2.write(str(raw) + '\n'+"\n")
    max_cnt = len(raw) #리스트 개수

    for i in range(0, max_cnt) : 
        content = str(raw[i]['content']) #텍스트
        offset = int(raw[i]['highlights'][0]['offset']) #오프셋
        length = int(raw[i]['highlights'][0]['length']) #해당 길이
        text = content[offset : offset + length] #부정 의미가 있는 문구 텍스트 추출
        sentiment=str(raw[i]['sentiment'])
        #print(text)
        file.write(str(text)+"\t"+str(sentiment) + '\n')

    n += 800
    m += 800

file.close()

02번쨰 기사입니다.손세희 한돈협회장이 임기를 함께할 회장단 및 이사회를 구성했다.?한돈협회는 오는 17일 제20대 회장 이취임식을 통해 손세희 號(호)의 출발을 알린다. 이에 앞서 손세희 회장은 임기 4년을 함께할 회장단 및 이사진을 구성했다.?부회장은 △조영욱 △구경본 △김춘일 △문석주 △고봉석 등 5명이 선임됐다. 이어 감사에는 왕영일, 이사에는 △최영길(경기도협의회장) △배상건(강원도협의회장) △이민영(충북도협의회장) △김은호(충남세종도협의회장) △이웅렬(전북도협의회장) △오재곤(전남도협의회장) △최재철(경남도협의회장) △유기옥(경남도협의회장) △김재우(제주도협의회장) △서정용 △오명준 △이재춘 △방희진 △이희영 △조상덕 △김현섭 △임성주 △이기홍 △오상도 △오승주 △주재용 △서부칠 △이재식 등을 선임했다.?  좌측부터 배병규 대의원회 감사, 배상건 대의원회 의장, 오재곤 선거관리위원장, 손세희 관리위원회 위원장, 이준길 관리위원회 위원(경기)한돈자조금관리위원회(이하 한돈자조금)가 한돈자조금 임원 선거 및 위원 보궐선거의 당선자를 확정하고 10일 당선증 전달식을 가졌다.부문별로 대의원회 의장에 배상건(강원 횡성·평창·영월·태백·삼척·정선·동해), 대의원회 감사에 이재식(경남 김해·창원)·배병규(충북 보은·영동·옥천), 관리위원회 위원장에 손세희(충남 홍성), 경기지역 위원 보궐선거에 이준길(경기 연천)씨가 각각 당선됐다.한돈자조금 대의원회 배상건 의장은 “한돈자조금 대의원은 전국의 한돈 농가로부터 위임받은 권한을 통해 한돈 산업과 농가를 대변해야 한다”며 “한돈 농가들과의 지속적인 커뮤니케이션을 이어


{
    "content": "02\ubc88\uca30 \uae30\uc0ac\uc785\ub2c8\ub2e4.\uc190\uc138\ud76c \ud55c\ub3c8\ud611\ud68c\uc7a5\uc774 \uc784\uae30\ub97c \ud568\uaed8\ud560 \ud68c\uc7a5\ub2e8 \ubc0f \uc774\uc0ac\u

{"document":{"sentiment":"neutral","confidence":{"negative":0.060888864,"positive":33.34781,"neutral":66.5913}},"sentences":[{"content":" 도체들이 어떤 등급에 가깝게 출하되었는지 평균 정보를 쉽게 확인할 수 있다.","offset":0,"length":43,"sentiment":"neutral","confidence":{"negative":2.5790735E-4,"positive":9.308501E-4,"neutral":0.99881124},"highlights":[{"offset":1,"length":41}]},{"content":" 해당 정보는 ‘거래증명통합포털(www.ekape.or.kr)’에서 전국 모든 농가가 직접 활용할 수 있다.","offset":43,"length":60,"sentiment":"neutral","confidence":{"negative":1.2429866E-4,"positive":2.539892E-4,"neutral":0.99962175},"highlights":[{"offset":38,"length":21}]},{"content":"장승진 원장은 “이번 서비스 제공을 통해 사육과 출하 시점 적정성 여부 등 농장관리를 위한 의사결정에 큰 도움이 될 것”이라며 “축평원은 앞으로도 축적해온 축산 빅데이터의 활용성을 높여 사회적 가치 제고에 힘쓰겠다”고 말했다.","offset":103,"length":126,"sentiment":"positive","confidence":{"negative":3.8548015E-4,"positive":0.9986035,"neutral":0.0010110417},"highlights":[{"offset":51,"length":15}]},{"content":"저작권자 ?","offset":229,"length":6,"sentiment":"neutral","confidence":{"negative"

{"document":{"sentiment":"negative","confidence":{"negative":68.17292,"positive":13.576349,"neutral":18.250729}},"sentences":[{"content":"따르면 올 1월~11월 12일 기준 야생 멧돼지 ASF 발생 건수는 780건으로 작년 동월 777건 대비 증가세로 돌아섰다.","offset":0,"length":69,"sentiment":"positive","confidence":{"negative":0.001663519,"positive":0.96788824,"neutral":0.030448187},"highlights":[{"offset":56,"length":12}]},{"content":" 특히 월별로 살펴보면 지난해 11월부터 이어진 ASF 발생 건수는 2월 167건을 최고점으로 3월부터 감소세로 돌아섰다.","offset":69,"length":68,"sentiment":"negative","confidence":{"negative":0.99734443,"positive":3.2863105E-4,"neutral":0.0023269476},"highlights":[{"offset":47,"length":20}]},{"content":" 이후 7월부터 다시 전년 대비 증가세로 전환, 11월 누적 집계 시 지난해 동월과 비교해 총 발생 건수를 추월했다.","offset":137,"length":65,"sentiment":"neutral","confidence":{"negative":2.1648275E-4,"positive":6.194151E-4,"neutral":0.9991641},"highlights":[{"offset":31,"length":33}]},{"content":" 이로써 국내 멧돼지에서의 ASF 발생은 총 1천700건을 넘어섰다.","offset":202,"length":38,"sentiment":"neutral","confidence":{"ne

{"document":{"sentiment":"negative","confidence":{"negative":66.669174,"positive":0.091592245,"neutral":33.23923}},"sentences":[{"content":"축산관련 규정 준수 특별점검이 명백한 과잉 행정규제라며, 항의하고, 즉각 중단할 것을 촉구했다.","offset":0,"length":53,"sentiment":"negative","confidence":{"negative":0.9988973,"positive":5.2054384E-4,"neutral":5.822075E-4},"highlights":[{"offset":41,"length":11}]},{"content":" 협회는 야생멧돼지의 남하는 한돈농가들의 잘못이 아닌 명백한 정부의 책임이자 잘못임에도 불구하고, 방역을 위한 가축전염병예방법 뿐만 아니라 축산법, 가축분뇨법, 악취방지법, 이력법 등 모든 축산관련 규정 준수여부를 전 농가를 대상으로 점검하고, 이를 처벌하고자 하는 것은 명백한 과잉 행정규제라고 반발했다.","offset":53,"length":171,"sentiment":"negative","confidence":{"negative":0.9987424,"positive":8.3685335E-4,"neutral":4.2078062E-4},"highlights":[{"offset":152,"length":18}]},{"content":"또한 야생멧돼지가 남하하여 철저한 차단방역이 이루어져야 하는 위태로운 현재 상황을 고려할 때 이러한 일제 점검 및 단속은 방역의 기본을 무시하는 행정규제라고 규정하고, 즉각 중단할 것을 강력 요구했다.?","offset":224,"length":113,"sentiment":"negative","confidence":{"negative":0.9989114,"positive":4.6670632E-4,"neutral":6.2194664E-4},"highlights":[{"o

{"document":{"sentiment":"positive","confidence":{"negative":15.017462,"positive":59.992786,"neutral":24.989752}},"sentences":[{"content":"했다.","offset":0,"length":3,"sentiment":"neutral","confidence":{"negative":8.561362E-4,"positive":8.446087E-4,"neutral":0.9982993},"highlights":[{"offset":0,"length":2}]},{"content":"그러나 올해 월별 삼겹 가격 분포도를 살펴보면 aT 기준 1년 내내 kg당 2만원 이상이 형성되고 있으며, 특히 6월부터 11월까지 2천500원 이상을 상회한 이후 삼겹 소비가 크게 줄었다.","offset":3,"length":106,"sentiment":"neutral","confidence":{"negative":7.470446E-4,"positive":2.2373507E-4,"neutral":0.99902916},"highlights":[{"offset":81,"length":24}]},{"content":"이에 따라 삼겹 재고가 증가하고 있으며, 덤핑 물량도 꾸준히 출현하고 있다.","offset":109,"length":42,"sentiment":"neutral","confidence":{"negative":1.8465453E-4,"positive":4.3110788E-4,"neutral":0.99938416},"highlights":[{"offset":23,"length":16}]},{"content":" 이 같이 현재 삼겹 가격은 소비 임계점을 넘은 상황으로, 김장철 소비 종료 이후 한돈 소비 침체에 뇌관이 될 수도 있을 것으로 보여, 업계가 우려하고 있다.","offset":151,"length":88,"sentiment":"negative","confidence":{"negati

1두=14.2% △18~19두=16.7% △16~17두=17.1% △16두 미만=32.5% 등으로 나타났다. 즉 국내 약 절반의 농가들이 MSY 18두 미만으로 나타났다. 특히 국제 경쟁력을 가진 MSY 22두 이상 농가 비율은 19.4%에 불과했다.2020년 PSY, MSY 성적별 농가수 비율이에 따라 돈가가 생산비 수준 및 이하 형성 시 MSY 18두 미만 농가들의 수익 악화로 경영 어려움이 가속화될 것으로 우려되고 있다. 특히 성적이 낮은 농가는 대부분 소규모 농장들로 돈가 하락 시 연쇄 도산이 우려되고 있어 생산성 제고가 시급한 것으로 분석되고 있다. 아울러 최근에는 생산비 증가에 따라 내년도에는 두당 손익이 더욱 감소할 것으로 보여, 생산성에 대한 중요성이 더욱 커질 것으로 전망되고 있다. 207번쨰 기사입니다.정부가 ‘포괄적 점진적 환태평양경제동반자협정(CPTPP)’ 가입 추진을 공식화하면서 농축산업계가 가입 철회를 주장하고 나섰다.홍남기 경제부총리 겸 기획재정부 장관은 지난 13일 대외경제관계장관회의에서 “교역과 투자 확대를 통한 경제적, 전략적 가치 등을 고려해 CPTPP 가입을 본격 추진한다”고 밝혔다. CPTPP는 미국이 TPP 탈퇴 후 일본, 호주, 멕시코 등 11개국이 18년 출범시킨 다자간 자유무역협정(FTA)으로 농축산업계 피해가 어느 무역협정보다 높을 것으로 우려되고 있다. CPTPP 관세 철폐율은 최대 96% 수준으로 시장 개방도가 높고 참여국 중 호주, 칠레, 캐나다 등 농업 수출국들이 많다. 특히 CPTPP 가입 시 동식물 위생?검역(SPS)의 구획화가 적용된다는 점이 가장 


{
    "content": "1\ub450=14.2% \u25b318~19\ub450=16.7% \u25b316~17\ub450=17.1% \u25b316\ub450 \ubbf8\ub9cc=32.5% \ub4f1\uc73c\ub85c \ub098\ud0c0\ub0ac\ub2e4. \uc989 \uad6d\ub0b4 \uc57d \uc808\ubc18




{
    "content": ""
}
Error : {"status":400,"error":{"errorCode":"E001","message":"Unsupported empty or blank text"}}


KeyError: 'sentences'

In [6]:
text

{'status': 400,
 'error': {'errorCode': 'E001', 'message': 'Unsupported empty or blank text'}}

In [9]:
week[207]['article'][0]

'정부가 ‘포괄적 점진적 환태평양경제동반자협정(CPTPP)’ 가입 추진을 공식화하면서 농축산업계가 가입 철회를 주장하고 나섰다.홍남기 경제부총리 겸 기획재정부 장관은 지난 13일 대외경제관계장관회의에서 “교역과 투자 확대를 통한 경제적, 전략적 가치 등을 고려해 CPTPP 가입을 본격 추진한다”고 밝혔다. CPTPP는 미국이 TPP 탈퇴 후 일본, 호주, 멕시코 등 11개국이 18년 출범시킨 다자간 자유무역협정(FTA)으로 농축산업계 피해가 어느 무역협정보다 높을 것으로 우려되고 있다. CPTPP 관세 철폐율은 최대 96% 수준으로 시장 개방도가 높고 참여국 중 호주, 칠레, 캐나다 등 농업 수출국들이 많다. 특히 CPTPP 가입 시 동식물 위생?검역(SPS)의 구획화가 적용된다는 점이 가장 우려되는 지점이다.이미 정부는 4개 분야에 있어서 국내 제도 정비를 통해 CPTPP 가입을 준비해왔으며 그 중 하나가 위생검역분야다. 얼마 전 정부가 개정 고시한 ‘지정 검역물의 수입에 관한 수입위험분석요령’을 통해 구획화 정의를 마련하고 교역 제한을 최소화하는 방향으로 수입위생조건을 정비했다. 구획이란 특정 질병에 대해 차별화된 동물 위생 상태를 가진 공통된 차단방역 관리체계에 의해 구분되는 동물 소집단으로 규정된다. 이에 질병 발생 여부를 따지는 단위가 한 국가나 지역이 아니라 개별 농장까지 좁혀질 수 있다.한국농업경영인중앙연합회는 “그동안 병해충, 가축질병 등을 이유로 수입을 규제해 온 생과실 및 신선 축산물의 국내 시장 진출이 가속화될 것”이라며 “한-미FTA 또는 한-중FTA를 넘어 기 체결한 어떠한 FTA보다 농업부문의 피해가 클 것”이라고 지적했다. 그러면서 “정부의 CPTPP 가입 선언은 대한민국 농업, 나아가 먹거리 주권 포기나 다름없다 할 것이다. 따라서 250만 농업인은 이를 저지하기 위해 대대적인 투쟁에 나설 것”이라고 경고했다.한편 CPTPP 가입을 위해서는 기존 회원국 만장일치 승인을 받아야 한다. 이에 국회 보고 절차를 거쳐 회원국들과 협상을 고

In [ ]:
text